In [ ]:
# Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive/')

# 目的の場所（フォルダ・ディレクトリ）へ移動（各自の環境で適宜修正）
%cd "/content/drive/MyDrive/Colab Notebooks/JKJ1A/"
%ls


---
### EX課題1


これまで説明した敵対的サンプルの生成方法（例えばFGSM）では，「正しいラベルでの誤差が大きくなるように」画像を変化させ敵対的サンプルを生成した．これはnon-targetedな方法（例えばnon-targeted FGSM）と呼ばれる．この場合，敵対的サンプルがどの誤ったラベルになるかは事前にわからない．

これに対してtargetedな方法は，「指定した誤ったラベルでの誤差が小さくなるように」画像を変化させる（正解ラベルに関する誤差は相対的に大きくなる）．Non-targetedな方法が実装できていれば，ほんの少し修正すればtargetedな方法に修正できる．Non-argeted I-FGSMを実装せよ．それを用いていくつか敵対的サンプルを生成し，うまくいっていることを確認せよ（練習問題１の様な感じで）．

授業ページの背景知識にある「敵対的サンプル入門」ではTargetedな方法で敵対的サンプルの生成を説明しているので適宜参考にせよ．
